In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!python3 -m pip install --user bert-tensorflow

     |████████████████████████████████| 71kB 27.3MB/s 


In [3]:
import os
import re
import time
import datetime
import glob
import pathlib

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from datetime import datetime
from collections import Counter
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_files
from tqdm import tqdm

print('Tensorflow version: {}'.format(tf.__version__))

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Tensorflow version: 1.14.0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10898407511658723436, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18344317799076504172
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18050405717902452905
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14892338381
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10752917613540893293
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [4]:
os.chdir('/content/gdrive/My Drive/bert/')
!pwd

/content/gdrive/My Drive/bert


In [0]:
import bert
from bert import run_classifier, optimization, tokenization, modeling

In [35]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip -O ./uncased_L-12_H-768_A-12.zip

--2019-07-08 05:16:39--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.197.176, 2404:6800:4004:80f::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.197.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘./uncased_L-12_H-768_A-12.zip’

./uncased_L-12_H-76 100%[===================>] 388.84M  61.0MB/s    in 6.1s    

2019-07-08 05:16:45 (63.7 MB/s) - ‘./uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [36]:
!unzip ./uncased_L-12_H-768_A-12.zip

Archive:  ./uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [37]:
!ls ./uncased_L-12_H-768_A-12/

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [0]:
!wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip -O ./model/wwm_uncased_L-24_H-1024_A-16.zip

['--2019-07-06 13:40:15--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip',
 'Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c01::80',
 'Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.',
 'HTTP request sent, awaiting response... 200 OK',
 'Length: 1248381879 (1.2G) [application/zip]',
 'Saving to: ‘./model/wwm_uncased_L-24_H-1024_A-16.zip’',
 '',
 '',
 '          ./model/w   0%[                    ]       0  --.-KB/s               ',
 '         ./model/ww   0%[                    ]   9.52M  47.4MB/s               ',
 '        ./model/wwm   1%[                    ]  23.66M  59.1MB/s               ',
 '       ./model/wwm_   3%[                    ]  38.29M  63.7MB/s               ',
 '      ./model/wwm_u   4%[                    ]  51.84M  64.8MB/s               ',
 '     ./model/wwm_un   5%[>                   ]  66.59M  66.5MB/s               ',
 '    .

In [0]:
!unzip ./model/wwm_uncased_L-24_H-1024_A-16.zip

Archive:  ./model/wwm_uncased_L-24_H-1024_A-16.zip
   creating: wwm_uncased_L-24_H-1024_A-16/
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_config.json  


In [6]:
!ls ./wwm_uncased_L-24_H-1024_A-16/

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [0]:
BERT_VOCAB= './uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = './uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = './uncased_L-12_H-768_A-12/bert_config.json'
OUTPUT_DIR = './output'

In [0]:
tokenization.validate_case_matches_checkpoint(True, BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [0]:
DATA_PATH = './data/train.csv'
data = pd.read_csv(DATA_PATH)

# label 1 for unreliable, label 0 for reliable
TEXT_COL = 'text'
LABEL_COL = 'label'

train_data, test_data = train_test_split(data, test_size=0.2)

In [10]:
train_data.head()

,id,title,author,text,label
11308,11308,Good Samaritan wearing Indian headdress disarm...,-NO AUTHOR-,Good Samaritan wearing Indian headdress disarm...,1
7991,7991,"Hillary Clinton, Donald Trump, Vladimir Putin:...",Andrea Kannapell and Sandra Stevenson,(Want to get this briefing by email? Here’s th...,0
3962,3962,Watch Ozzy Man’s incredibly sweary review of t...,OK,Next Swipe left/right Watch Ozzy Man’s incredi...,1
16766,16766,NASA Actually Recorded Sound In Space. What Yo...,Dikran Arakelian (noreply@blogger.com),Share on Facebook NASA Space Sounds: what happ...,1
11461,11461,Norwegian Government to Deport White Patriot W...,Red Ice,Norwegian Government to Deport White Patriot W...,1


In [0]:
def create_examples(df):
    """Creates examples for the training and dev sets."""
    examples = []
    for index, value in df.iterrows():
        text_a = str(value['text'])
        label = (value['label'])
        examples.append(
            bert.run_classifier.InputExample(guid=None, text_a=text_a,
                                             text_b=None, label=label))
    return examples

In [0]:
TRAIN_VAL_RATIO = 0.8
LENGTH = train_data.shape[0]
TRAIN_SIZE = int(TRAIN_VAL_RATIO*LENGTH)

In [0]:
train_examples = create_examples(train_data)
test_examples = create_examples(test_data)

In [0]:
MAX_SEQ_LENGTH = 128
label_list = [0, 1]

train_features = bert.run_classifier.convert_examples_to_features(train_examples, 
                                                                  label_list,
                                                                  MAX_SEQ_LENGTH, 
                                                                  tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_examples, 
                                                                  label_list,
                                                                  MAX_SEQ_LENGTH, 
                                                                  tokenizer)

In [96]:
len(train_features), len(test_features)

(16640, 4160)

In [0]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 4

WARMUP_PROPORTION = 0.1

SAVE_CHECKPOINT_STEPS = 4000
SAVE_SUMMARY_STEPS = 1000

In [98]:
num_train_steps = int(len(train_features)/ BATCH_SIZE * EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print("***** Running training *****")
print("  Num of training features = %d" % len(train_features))
print("  Batch size = %d" % BATCH_SIZE)
print("  Num steps = %d"% num_train_steps)

***** Running training *****
  Num of training features = 16640
  Batch size = 32
  Num steps = 2080


In [0]:
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINT_STEPS
)

In [100]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
bert_config.__dict__

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [0]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings=True):
    """Creates a classification model."""
    model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=True)

    output_layer = model.get_pooled_output()
    hidden_size = output_layer.shape[-1].value
    print('Hidden size: ',hidden_size)
    
    output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())
    
    with tf.variable_scope('loss'):
        # fine-tuning last layer
        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        
        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        
        predicted = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        
        if is_training:
            per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
            loss = tf.reduce_mean(per_example_loss)
            return loss, predicted, log_probs
            
            
        elif not is_training:
            predicted = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
            return predicted, log_probs

In [0]:
def model_fn_builder(bert_config, init_checkpoint, num_labels, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    # Wraps model_fn as Estimator
    def model_fn(features, labels, mode, params):
        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))
            
        input_ids = features['input_ids']
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        
        is_training = (mode == tf.estimator.ModeKeys.TRAIN or 
                       mode == tf.estimator.ModeKeys.EVAL)
            
        # Training or evaluating
        if is_training:
            (total_loss, predicted_labels, log_probs) = create_model(bert_config, is_training,
                                                              input_ids, input_mask,
                                                              segment_ids, label_ids,
                                                              num_labels)
            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(
                    label_ids,
                    predicted_labels)
                auc = tf.metrics.auc(
                    label_ids,
                    predicted_labels)
                recall = tf.metrics.recall(
                    label_ids,
                    predicted_labels)
                precision = tf.metrics.precision(
                    label_ids,
                    predicted_labels) 
                true_pos = tf.metrics.true_positives(
                    label_ids,
                    predicted_labels)
                true_neg = tf.metrics.true_negatives(
                    label_ids,
                    predicted_labels)   
                false_pos = tf.metrics.false_positives(
                    label_ids,
                    predicted_labels)  
                false_neg = tf.metrics.false_negatives(
                    label_ids,
                    predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }
            
            optimizer = bert.optimization.create_optimizer(total_loss, learning_rate,
                                                          num_train_steps, num_warmup_steps,
                                                          use_tpu=False)
            
            tvars = tf.trainable_variables()
            initialized_variable_names = {}
            scaffold_fn = None
            if init_checkpoint:
                (assignment_map, initialized_variable_names
                ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
                
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                
            tf.logging.info("**** Trainable Variables ****")
            for var in tvars:
                init_string = ''
                if var.name in initialized_variable_names:
                    init_string = ', *INIT_FROM_CKPT*'
                tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)
            
            if mode == tf.estimator.ModeKeys.TRAIN:
                output_spec = tf.estimator.EstimatorSpec(
                                mode=mode,
                                loss=total_loss,
                                train_op=optimizer)
                
            elif mode == tf.estimator.ModeKeys.EVAL:
                eval_metrics = metric_fn(label_ids, predicted_labels)
                output_spec = tf.estimator.EstimatorSpec(
                                mode=mode,
                                loss=total_loss,
                                eval_metric_ops=eval_metrics)
                
        # predicting
        elif not is_training:
            (predicted_labels, log_probs) = create_model(bert_config, is_training,
                                                          input_ids, input_mask,
                                                          segment_ids, label_ids,
                                                          num_labels)
            output_spec = tf.estimator.EstimatorSpec(
                                mode=mode,
                                predictions={
                                    'probabilities': log_probs,
                                    'labels': predicted_labels
                                })
    
        print(output_spec)
        return output_spec

    return model_fn

In [0]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(bert_config=bert_config,
                            num_labels=len(label_list),
                            init_checkpoint=BERT_INIT_CHKPNT,
                            learning_rate=LEARNING_RATE,
                            num_train_steps=num_train_steps,
                            num_warmup_steps=num_warmup_steps,
                            use_tpu=False,
                            use_one_hot_embeddings=True
                           )

estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={"batch_size": BATCH_SIZE}
)

In [0]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False
)

## Train with estimator

In [107]:
print('Begin training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('Training took time ', datetime.now() - current_time)

Begin training!
Training took time  0:00:00.022142


## Getting the last 50 examples in test_examples as predict_examples


In [0]:
test_examples = test_examples[:-50]
predict_examples = test_examples[-50:]

test_features = bert.run_classifier.convert_examples_to_features(test_examples, 
                                                                  label_list,
                                                                  MAX_SEQ_LENGTH, 
                                                                  tokenizer)

## Evaluate test_examples with trained estimator

In [199]:
test_input_fn = bert.run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

estimator.evaluate(input_fn=test_input_fn, steps=None)

Hidden size:  768
EstimatorSpec(mode='eval', predictions={}, loss=<tf.Tensor 'loss/Mean:0' shape=() dtype=float32>, train_op=None, eval_metric_ops={'eval_accuracy': (<tf.Tensor 'accuracy/value:0' shape=() dtype=float32>, <tf.Tensor 'accuracy/update_op:0' shape=() dtype=float32>), 'f1_score': (<tf.Tensor 'f1/Max:0' shape=() dtype=float32>, <tf.Tensor 'f1/Max_1:0' shape=() dtype=float32>), 'auc': (<tf.Tensor 'auc/value:0' shape=() dtype=float32>, <tf.Tensor 'auc/update_op:0' shape=() dtype=float32>), 'precision': (<tf.Tensor 'precision/value:0' shape=() dtype=float32>, <tf.Tensor 'precision/update_op:0' shape=() dtype=float32>), 'recall': (<tf.Tensor 'recall/value:0' shape=() dtype=float32>, <tf.Tensor 'recall/update_op:0' shape=() dtype=float32>), 'true_positives': (<tf.Tensor 'true_positives/Identity:0' shape=() dtype=float32>, <tf.Tensor 'true_positives/AssignAdd:0' shape=() dtype=float32_ref>), 'true_negatives': (<tf.Tensor 'true_negatives/Identity:0' shape=() dtype=float32>, <tf.Ten

{'auc': 0.9914619,
 'eval_accuracy': 0.99148417,
 'f1_score': 0.9912695,
 'false_negatives': 19.0,
 'false_positives': 16.0,
 'global_step': 4160,
 'loss': 0.056471173,
 'precision': 0.99201196,
 'recall': 0.9905284,
 'true_negatives': 2088.0,
 'true_positives': 1987.0}

## Predict function

In [0]:
def predict(examples):
    predict_features = bert.run_classifier.convert_examples_to_features(examples, 
                                                                    label_list,
                                                                    MAX_SEQ_LENGTH, 
                                                                    tokenizer)
    predict_input_fn = bert.run_classifier.input_fn_builder(features=predict_features,
                                                    seq_length=MAX_SEQ_LENGTH,
                                                    is_training=False,
                                                    drop_remainder=False)
    predicted = list(estimator.predict(predict_input_fn))
    predictions = []
    
    examples = list(examples)
    for i in range(len(examples)):
        text = examples[i].text_a
        real_label = 'unreliable' if predict_features[i].label_id == 1 else 'reliable'
        predicted_label = 'unreliable' if predicted[i]['labels'] == 1 else 'reliable'
        probs = predicted[i]['probabilities']
        predictions.append({'text': text,'probs': probs, 'predicted': predicted_label, 'real': real_label})
    return predictions

In [224]:
predictions = predict(predict_examples)

Hidden size:  768
EstimatorSpec(mode='infer', predictions={'probabilities': <tf.Tensor 'loss/LogSoftmax:0' shape=(?, 2) dtype=float32>, 'labels': <tf.Tensor 'loss/Squeeze_1:0' shape=<unknown> dtype=int32>}, loss=None, train_op=None, eval_metric_ops={}, export_outputs={'serving_default': <tensorflow.python.saved_model.model_utils.export_output.PredictOutput object at 0x7fe994fe4438>}, training_chief_hooks=(), training_hooks=(), scaffold=<tensorflow.python.training.monitored_session.Scaffold object at 0x7fe994ff6da0>, evaluation_hooks=(), prediction_hooks=())


In [232]:
for item in predictions:
    print('PROBS: {}, PREDICTED: {}, REAL: {}\n'.format(item['probs'], item['predicted'], item['real']))
    

PROBS: [-2.6106494e-05 -1.0553666e+01], PREDICTED: reliable, REAL: reliable

PROBS: [-1.1055892e+01 -1.5735503e-05], PREDICTED: unreliable, REAL: unreliable

PROBS: [-1.0894303e+01 -1.8596476e-05], PREDICTED: unreliable, REAL: unreliable

PROBS: [-2.7894584e-05 -1.0486924e+01], PREDICTED: reliable, REAL: reliable

PROBS: [-1.10946045e+01 -1.51394652e-05], PREDICTED: unreliable, REAL: unreliable

PROBS: [-3.4331686e-05 -1.0278375e+01], PREDICTED: reliable, REAL: reliable

PROBS: [-2.5510462e-05 -1.0577895e+01], PREDICTED: reliable, REAL: reliable

PROBS: [-2.5748875e-05 -1.0566983e+01], PREDICTED: reliable, REAL: reliable

PROBS: [-1.1079879e+01 -1.5377880e-05], PREDICTED: unreliable, REAL: unreliable

PROBS: [-1.1046756e+01 -1.5973917e-05], PREDICTED: unreliable, REAL: unreliable

PROBS: [-4.0768748e-05 -1.0108087e+01], PREDICTED: reliable, REAL: reliable

PROBS: [-1.1072077e+01 -1.5497088e-05], PREDICTED: unreliable, REAL: unreliable

PROBS: [-1.1075552e+01 -1.5497088e-05], PREDICTED: